# Task 2: User Engagement Analysis

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file

In [7]:
load_dotenv('../venv/venv')

True

# Retrieve database connection details from environment variables

In [8]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
table_name = 'xdr_data'

# Create the connection string

In [ ]:
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

# Query the data

In [ ]:
query = 'SELECT * FROM xdr_data'
data = pd.read_sql(query, engine)

# Aggregate Metrics per Customer ID (MSISDN)

In [ ]:
# Define DL and UL columns
dl_columns = [
    'Social Media DL (Bytes)', 
    'Google DL (Bytes)', 
    'Email DL (Bytes)', 
    'Youtube DL (Bytes)', 
    'Netflix DL (Bytes)', 
    'Gaming DL (Bytes)', 
    'Other DL (Bytes)'
]

ul_columns = [
    'Social Media UL (Bytes)', 
    'Google UL (Bytes)', 
    'Email UL (Bytes)', 
    'Youtube UL (Bytes)', 
    'Netflix UL (Bytes)', 
    'Gaming UL (Bytes)', 
    'Other UL (Bytes)'
]

agg_data = data.groupby('MSISDN/Number').agg(
    sessions_frequency=('Bearer Id', 'count'),
    total_duration=('Dur. (ms)', 'sum'),
    **{col: (col, 'sum') for col in dl_columns},  # Sum each DL column
    **{col: (col, 'sum') for col in ul_columns}   # Sum each UL column
)

# Calculate total data volume
agg_data['total_traffic'] = agg_data[dl_columns].sum(axis=1) + agg_data[ul_columns].sum(axis=1)

# Drop the intermediary columns
agg_data = agg_data.drop(columns=dl_columns + ul_columns)

# Top 10 customers per engagement metric
top_sessions = agg_data.nlargest(10, 'sessions_frequency')
top_duration = agg_data.nlargest(10, 'total_duration')
top_traffic = agg_data.nlargest(10, 'total_traffic')

print(top_sessions)
print(top_duration)
print(top_traffic)

# Aggregate Metrics per Customer ID (MSISDN)

In [ ]:
# Define DL and UL columns
dl_columns = [
    'Social Media DL (Bytes)', 
    'Google DL (Bytes)', 
    'Email DL (Bytes)', 
    'Youtube DL (Bytes)', 
    'Netflix DL (Bytes)', 
    'Gaming DL (Bytes)', 
    'Other DL (Bytes)'
]

ul_columns = [
    'Social Media UL (Bytes)', 
    'Google UL (Bytes)', 
    'Email UL (Bytes)', 
    'Youtube UL (Bytes)', 
    'Netflix UL (Bytes)', 
    'Gaming UL (Bytes)', 
    'Other UL (Bytes)'
]

agg_data = data.groupby('MSISDN/Number').agg(
    sessions_frequency=('Bearer Id', 'count'),
    total_duration=('Dur. (ms)', 'sum'),
    **{col: (col, 'sum') for col in dl_columns},  # Sum each DL column
    **{col: (col, 'sum') for col in ul_columns}   # Sum each UL column
)

# Calculate total data volume
agg_data['total_traffic'] = agg_data[dl_columns].sum(axis=1) + agg_data[ul_columns].sum(axis=1)

# Drop the intermediary columns
agg_data = agg_data.drop(columns=dl_columns + ul_columns)

# Top 10 customers per engagement metric
top_sessions = agg_data.nlargest(10, 'sessions_frequency')
top_duration = agg_data.nlargest(10, 'total_duration')
top_traffic = agg_data.nlargest(10, 'total_traffic')

print("Top 10 by Sessions Frequency:\n", top_sessions)
print("\nTop 10 by Total Duration:\n", top_duration)
print("\nTop 10 by Total Traffic:\n", top_traffic)

# Aggregate User Total Traffic per Application

In [ ]:
# Calculate total traffic per application for each user
app_traffic = data.groupby('MSISDN/Number').agg({
    'Social Media DL (Bytes)': 'sum',
    'Youtube DL (Bytes)': 'sum',
    'Netflix DL (Bytes)': 'sum',
    'Google DL (Bytes)': 'sum',
    'Email DL (Bytes)': 'sum',
    'Gaming DL (Bytes)': 'sum',
    'Other DL (Bytes)': 'sum'
})

# Top 10 most engaged users per application
top_social_media = app_traffic.nlargest(10, 'Social Media DL (Bytes)')
top_youtube = app_traffic.nlargest(10, 'Youtube DL (Bytes)')
top_netflix = app_traffic.nlargest(10, 'Netflix DL (Bytes)')

print("Top 10 by Social Media Traffic:\n", top_social_media)
print("\nTop 10 by YouTube Traffic:\n", top_youtube)
print("\nTop 10 by Netflix Traffic:\n", top_netflix)

# Normalize Engagement Metrics

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
normalized_data = scaler.fit_transform(agg_data)

# K-Means Clustering (k=3)

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
agg_data['cluster'] = kmeans.fit_predict(normalized_data)

# Metrics for Each Cluster

In [ ]:
cluster_metrics = agg_data.groupby('cluster').agg(
    min_sessions=('sessions_frequency', 'min'),
    max_sessions=('sessions_frequency', 'max'),
    avg_sessions=('sessions_frequency', 'mean'),
    total_sessions=('sessions_frequency', 'sum'),
    min_duration=('total_duration', 'min'),
    max_duration=('total_duration', 'max'),
    avg_duration=('total_duration', 'mean'),
    total_duration=('total_duration', 'sum'),
    min_traffic=('total_traffic', 'min'),
    max_traffic=('total_traffic', 'max'),
    avg_traffic=('total_traffic', 'mean'),
    total_traffic=('total_traffic', 'sum')
)

print(cluster_metrics)

# Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.barplot(x=cluster_metrics.index, y=cluster_metrics['avg_sessions'])
plt.title('Average Sessions per Cluster')
plt.show()

# Elbow Method to Optimize k

In [ ]:
import matplotlib.pyplot as plt

inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(normalized_data)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia, marker='o')
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

# Aggregate User Total Traffic per Application

In [ ]:
app_traffic = data[['Social Media DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 
                    'Google DL (Bytes)', 'Email DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']].sum()

top_apps = app_traffic.nlargest(3)

print(top_apps)

# Plot Top 3 Applications 

In [ ]:
top_apps.plot(kind='bar')
plt.title('Top 3 Most Used Applications')
plt.ylabel('Total Traffic (Bytes)')
plt.show()